# **Tarefa: Clustering**

### Nesta tarefa, você deve carregar um dataset e agrupar os objetos utilizando algoritmos de pelo menos duas categorias diferentes (centroide, hierárquico, densidade)

**Dicas:**
- Explore diferentes parâmetros, como o número de clusters ou a distância usada, para observar como isso afeta os resultados.
- Analise os insights que surgem a partir das visualizações dos agrupamentos encontrados.

# **Sobre o Dataset**

Este dataset contém informações detalhadas sobre clientes, como dados demográficos, hábitos de compra e interações com campanhas de marketing. Ele permite identificar perfis de clientes e realizar segmentações com base em suas preferências e comportamentos de consumo.

**Features:**
- **Year_Birth:** Ano de nascimento do indivíduo.
- **Education:** Nível mais alto de educação alcançado.
- **Marital_Status:** Estado civil do indivíduo.
- **Income:** Renda anual do indivíduo.
- **Kidhome:** Número de crianças pequenas na casa.
- **Teenhome:** Número de adolescentes na casa.
- **Recency:** Dias desde a última compra ou interação.
- **MntWines:** Quantia gasta em vinhos.
- **MntFruits:** Quantia gasta em frutas.
- **MntMeatProducts:** Quantia gasta em produtos de carne.
- **MntFishProducts:** Quantia gasta em produtos de peixe.
- **MntSweetProducts:** Quantia gasta em doces.
- **NumStorePurchases:** Número de compras em lojas físicas.
- **NumWebVisitsMonth:** Número de visitas ao site da empresa em um mês.


# **Carregue as bibliotecas**

Dica: Para esta tarefa, além das bibliotecas `pandas` e `matplotlib`, explore funções das bibliotecas `sklearn` e `sklearn_extra`. Considere os seguintes pacotes:
- **cluster:** KMeans, KMedoids, AgglomerativeClustering, DBSCAN
- **preprocessing:** MinMaxScaler (normalização) e OneHotEncoder (binarização)
- **metrics:** silhouette_score

Sinta-se à vontade para adicionar outros pacotes que achar úteis.

```python
!pip install scikit-learn-extra
from sklearn_extra.cluster import KMedoids
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN


In [131]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.cluster import DBSCAN
from sklearn_extra.cluster import 

### **Carregue os Dados**

In [132]:
path = 'customer_segmentation-24-2.csv'
df = pd.read_csv(path)
df.head()

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,NumStorePurchases,NumWebVisitsMonth
0,1957,Graduation,Single,58138.0,0,0,58,635,88,546,172,88,4,7
1,1954,Graduation,Single,46344.0,1,1,38,11,1,6,2,1,2,5
2,1965,Graduation,Together,71613.0,0,0,26,426,49,127,111,21,10,4
3,1984,Graduation,Together,26646.0,1,0,26,11,4,20,10,3,4,6
4,1981,PhD,Married,58293.0,1,0,94,173,43,118,46,27,6,5


In [133]:
print(df.Education.unique())
print(df.Marital_Status.unique())
print(df.isnull().sum())

['Graduation' 'PhD' 'Master' 'Basic' '2n Cycle']
['Single' 'Together' 'Married' 'Divorced' 'Widow']
Year_Birth            0
Education             0
Marital_Status        0
Income               13
Kidhome               0
Teenhome              0
Recency               0
MntWines              0
MntFruits             0
MntMeatProducts       0
MntFishProducts       0
MntSweetProducts      0
NumStorePurchases     0
NumWebVisitsMonth     0
dtype: int64


## **Pré-Processamento dos Dados**

Aplique as técnicas de pré-processamento que vimos em aula no dataset fornecido.

### **Dicas:**
- **Converter dados categóricos para numéricos:** Use `LabelEncoder` ou `OneHotEncoder`.

- **Transformar os dados:** Experimente `MinMaxScaler` para ajustar a escala dos dados.

- **Remover outliers:** Verifique outliers com base no IQR e os elimine se necessário.

- **Tratar valores nulos:** Preencha com a média, moda, mediana ou remova as linhas, dependendo do impacto no dataset.


In [134]:
oe_education = OrdinalEncoder(categories=[['Basic', 'Graduation', '2n Cycle', 'Master', 'PhD']])
df['Education'] = oe_education.fit_transform(df[['Education']])

ohe = OneHotEncoder(sparse_output=False)
marital_status = ohe.fit_transform(df[['Marital_Status']])
dfMarital_Status = pd.DataFrame(marital_status, columns=ohe.get_feature_names_out(['Marital_Status']))
df = pd.concat([df, dfMarital_Status], axis = 1)
df.drop(columns=['Marital_Status'], axis = 1, inplace = True)
df.reset_index(drop=True, inplace=True)

# TODO: should we input the mean in the null values?
df['Income'] = df['Income'].fillna(df['Income'].mean())

# TODO: analyze outliers and decide which ones to remove
for column in []:
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    df = df[(df[column] < (Q3 + 1.5 * IQR)) & (df[column] > (Q1 - 1.5 * IQR))]

scaler = MinMaxScaler()
df[df.columns] = scaler.fit_transform(df[df.columns])

df.head()

,Year_Birth,Education,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,NumStorePurchases,NumWebVisitsMonth,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Marital_Status_Together,Marital_Status_Widow
0,0.621359,0.25,0.348178,0.0,0.0,0.585859,0.425603,0.442211,0.316125,0.677165,0.334601,0.307692,0.35,0.0,0.0,1.0,0.0,0.0
1,0.592233,0.25,0.274442,0.5,0.5,0.383838,0.007373,0.005025,0.002900,0.007874,0.003802,0.153846,0.25,0.0,0.0,1.0,0.0,0.0
2,0.699029,0.25,0.432423,0.0,0.0,0.262626,0.285523,0.246231,0.073086,0.437008,0.079848,0.769231,0.20,0.0,0.0,0.0,1.0,0.0
3,0.883495,0.25,0.151291,0.5,0.0,0.262626,0.007373,0.020101,0.011021,0.039370,0.011407,0.307692,0.30,0.0,0.0,0.0,1.0,0.0
4,0.854369,1.00,0.349147,0.5,0.0,0.949495,0.115952,0.216080,0.067865,0.181102,0.102662,0.461538,0.25,0.0,1.0,0.0,0.0,0.0


# **Clustering**

**Escolha ao menos 2 algoritmos de clustering que pertençam a métodos diferentes (centróides, densidade ou hierárquico).**

### **Passos:**

1. Para cada algoritmo, execute-o em 6 diferentes cenários, variando os parâmetros.

2. Calcule o índice de silhueta `silhouette_score` para cada cenários e compare os resultados.

### **Resultados**

1. **Resumo dos Resultados:**
   - Para cada algoritmo utilizado, apresente os resultados dos 6 cenários testados e compare as métricas calculadas.

2. **Visualização Gráfica:**
   - Plote um gráfico comparando o índice de silhueta entre os cenários.





## **Encontre o melhor clustering para cada um dos algoritmos selecionados**

Depois de executar os algoritmos com diferentes combinações de parâmetros, identifique quais obtiveram os melhores resultados de acordo com índice de silhueta para cada um dos 2 (ou mais) algoritmos selecionados.


## **Para cada algoritmo, visualize graficamente o modelo que teve o melhor desempenho**

Com base nos modelos de melhor desempenho, escolha variáveis e crie gráficos de dispersão que permitam visualizar os clusters gerados.

**Dica:**

Ao criar scatter plots, teste combinações de 2 variáveis do dataset para facilitar a visualização dos clusters. Para o clustering hierárquico, experimente também plotar um dendrograma.

### **Dendrograma:**
Use o código abaixo para o dendrograma:
```python
import scipy.cluster.hierarchy as sch

Z = sch.linkage(df[[variavel_1, variavel_2]], method='centroid')

# p = altura máxima que será exibida
sch.dendrogram(Z, p = ?, truncate_mode = "level") # Mude o valor de p para melhorar a vizualização do seu dendrograma

```



[
Documentação Dendrograma](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html#scipy.cluster.hierarchy.dendrogram)

### **Análise dos Clusters**

Após identificar o melhor modelo, selecione **3 itens de cada cluster** e explique o que cada agrupamento parece representar com base nas características dos itens.

1. **Análise dos Itens:**
   - Analise as características principais desses itens (ex.: renda, número de compras, produtos comprados).
   - Identifique padrões ou características que diferenciam cada cluster.

2. **Interpretação dos Agrupamentos:**
   - Explique o que cada cluster parece estar representando.
   - Relacione as características predominantes de cada cluster a possíveis perfis de clientes (ex.: clientes com alta renda e foco em produtos premium, ou clientes com baixa frequência de compras).


Responda aqui....